# Continuous Tomography

This Juypter Notebook is a tutorial for generating simulated data for the publication: ..................

Table of Contents


## 1. Manage Imports 
This section is to manage imports for libraries necessary to run the code. Data will be simulated from the Nanocage 3D volume.

In [ ]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes


from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()


In [36]:
def quantify_backlash(scheme, n_angles, cumulation):
    backlash = 2
    angles = np.array([scheme.get_angle() for i in range(n_angles)])
    added_backlash = 0 
    for i, angle in enumerate(angles):
        if i ==0:
            pass
        elif angle<angles[i-1]:
            angles[i] -= backlash
            added_backlash += backlash
    cumulative_backlash = cumulation + added_backlash
    index = n_angles
    
    return index, added_backlash, cumulative_backlash

def process_with_backlash(scheme, n_angles):
    backlash = 2
    angles = np.array([scheme.get_angle() for i in range(n_angles)])
    for i, angle in enumerate(angles):
        if i ==0:
            pass
        elif angle<angles[i-1]:
            angles[i] -= backlash
    ts = tomobase.processes.project(vol, angles)
    ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 0, inplace=False)
    rec = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
    ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
    psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
    index = n_angles
    
    return index, ssim, psnr

In [ ]:
def process_calculate_psnr_ssim(scheme, n_angles):
        angles = np.array([scheme.get_angle() for j in range(n_angles)])
        ts = tomobase.processes.project(vol, angles)
        ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 0, inplace=False)
        rec = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
        ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
        index = n_angles

        return index, psnr, ssim

## Processes
### Incremental Scanning

The following is a script for running a process for incremental scanning

In [ ]:
schemes = [tomobase.tiltschemes.Incremental(-64, 64, 32), 
           tomobase.tiltschemes.Incremental(-64, 64, 16),
           tomobase.tiltschemes.Incremental(-64, 64, 8),
           tomobase.tiltschemes.Incremental(-64, 64, 4),
           tomobase.tiltschemes.Incremental(-64, 64, 2)]

n_angles = [5, 9, 17, 33, 65]
array = np.zeros((len(schemes), 3))
for i, item in enumerate(schemes):
    array[i,0], array[i,1], array[i,2] = process_calculate_psnr_ssim(item, n_angles[i])
    
for i, item in enumerate(schemes):
    print(array[i,0], array[i,1], array[i,2])

### GRS

Loop for performing GRS acquisition and analysis

In [37]:
num = 65
ssim = [None]*num
psnr = [None]*num
index = [None]*num

cumulation = 0
array = np.zeros((num, 3))
for i in range(num):
    scheme = tomobase.tiltschemes.GRS(-64, 64)
    angles = np.array([scheme.get_angle() for j in range(i+1)])
    array[i,0], array[i,1], array[i,2] = quantify_backlash(scheme, i+1, cumulation)
    cumulation = array[i,2]
        
for i in range(len(index)):    
    print(array[i,0], array[i,1], array[i,2])

1.0 0.0 0.0
2.0 2.0 2.0
3.0 2.0 4.0
4.0 2.0 6.0
5.0 6.0 12.0
6.0 6.0 18.0
7.0 8.0 26.0
8.0 8.0 34.0
9.0 10.0 44.0
10.0 12.0 56.0
11.0 12.0 68.0
12.0 12.0 80.0
13.0 16.0 96.0
14.0 16.0 112.0
15.0 18.0 130.0
16.0 18.0 148.0
17.0 20.0 168.0
18.0 22.0 190.0
19.0 22.0 212.0
20.0 24.0 236.0
21.0 24.0 260.0
22.0 26.0 286.0
23.0 28.0 314.0
24.0 28.0 342.0
25.0 28.0 370.0
26.0 32.0 402.0
27.0 32.0 434.0
28.0 34.0 468.0
29.0 34.0 502.0
30.0 36.0 538.0
31.0 38.0 576.0
32.0 38.0 614.0
33.0 38.0 652.0
34.0 42.0 694.0
35.0 42.0 736.0
36.0 44.0 780.0
37.0 44.0 824.0
38.0 44.0 868.0
39.0 48.0 916.0
40.0 48.0 964.0
41.0 50.0 1014.0
42.0 50.0 1064.0
43.0 52.0 1116.0
44.0 54.0 1170.0
45.0 54.0 1224.0
46.0 54.0 1278.0
47.0 58.0 1336.0
48.0 58.0 1394.0
49.0 60.0 1454.0
50.0 60.0 1514.0
51.0 62.0 1576.0
52.0 64.0 1640.0
53.0 64.0 1704.0
54.0 66.0 1770.0
55.0 66.0 1836.0
56.0 68.0 1904.0
57.0 70.0 1974.0
58.0 70.0 2044.0
59.0 70.0 2114.0
60.0 74.0 2188.0
61.0 74.0 2262.0
62.0 76.0 2338.0
63.0 76.0 2414.0
64.

### Binary

In [38]:
num = 65
array = np.full((num, 3), np.nan)
for i in range(num):
    scheme = tomobase.tiltschemes.Binary(-64, 64, 8)
    angles = np.array([scheme.get_angle() for j in range(i+1)])

    full_set =False
    if i >= 2:
        if (angles[-1]>angles[-2]) and (angles[-2]<angles[-3]):
            full_set = True
        if (angles[-1]<angles[-2]) and (angles[-2]>angles[-3]):
            full_set = True
    
    if full_set:
        angles = np.array([scheme.get_angle() for j in range(i)])
        array[i,0], array[i,1], array[i,2] = process_with_backlash(scheme, i) 
        
    
array = array[~np.isnan(array).any(axis=1)]
for i in range(array.shape[0]):    
    print(array[i,0], array[i,1], array[i,2])

2024-11-26 17:03:58,268 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:04:26,257 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:04:54,481 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:05:21,693 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:05:50,278 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:06:20,203 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>
2024-11-26 17:06:51,888 - IN

9.0 0.7435413057751641 17.06704126644558
18.0 0.8004658461234208 20.848143352784803
26.0 0.8240465041244672 22.086736993710577
35.0 0.8186943875196561 21.57461082285444
43.0 0.8243417110138602 21.80955610517958
52.0 0.8418667458636405 22.528834086732545
60.0 0.8378206317040944 22.391449928791282
